##### Loading Dependencies

In [20]:
import pandas as pd
import tensorflow as tf
from IPython import display
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import plot_model

Importing the data: